In [1]:
import api_scraper
api_data = api_scraper.get_water_obs_links()

In [2]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import optimize
from scipy.optimize import curve_fit
import numpy as np
from scipy import optimize
import pandas as pd
import dateutil.parser as date_parser
import datetime

In [3]:
sensors = pd.DataFrame(api_data)
sensors

,coords,desc,elev,link,name
0,"[-81.07081, 31.98277]",Skidaway Road at Herb River Sea Level Sensor,2.36,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-031
1,"[-81.084099, 32.085707]",Hutchinson Island environmental sensors,2.8,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-005
2,"[-81.201916, 31.791509]",Kilkenny Creek environmental sensors,2.7,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-004
3,"[-80.959417, 32.034717]",Bull River Marina environmental sensors,2.0,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-011
4,"[-81.153906, 31.935783]",Coffee Bluff Marina environmental sensors,2.4,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-001
5,"[-81.063662, 32.000149]",LaRoche Avenue at Nottingham Creek Sea Level S...,2.38,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-032
6,"[-81.021504, 31.990396]",Skidaway Dock environmental sensors,3.103,https://api.sealevelsensors.org/v1.0/Datastrea...,esp32-rn2903-scott-01
7,"[-80.866956, 32.006419]",Catalina Drive environmental sensors,2.23,https://api.sealevelsensors.org/v1.0/Datastrea...,esp32-rn2903-04
8,"[-81.08402, 31.92835]",Faye Drive on Burnside Island Sea Level Sensor,3.66,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-041
9,"[-80.962163, 32.018171]",Walthour Road environmental sensors,2.83,https://api.sealevelsensors.org/v1.0/Datastrea...,gt-envsense-009


In [8]:
def curve_equation(x, amp1, amp2, periodicity, xshift, yshift):
    return (amp1 * np.sin(periodicity * (x + xshift))) + (amp2 * np.sin(periodicity / 2 * (x + xshift))) + yshift

#current_date is a datetime object
def get_train_test(current_date, sensor_desc, train_delta = 8, test_delta = 1):
    link = sensors[sensors["desc"] == sensor_desc]["link"].iloc[0]
    elevation = sensors[sensors["desc"] == sensor_desc]["elev"].iloc[0]
    
    eight_days = datetime.timedelta(days=train_delta)
    one_day = datetime.timedelta(days=test_delta)
    
    start_date_train = (current_date - eight_days).isoformat() + "Z"
    end_date_train = (current_date - one_day).isoformat() + "Z"
    start_date_test = (current_date - one_day).isoformat() + "Z"
    end_date_test = (current_date).isoformat() + "Z"
    
    train = api_scraper.get_obs_for_link(link, start_date_train, end_date_train)
    train = pd.DataFrame(train, columns= ["value", "timestamp"])
    # train["parsed_timestamp"] = train["timestamp"].map(lambda date: date_parser.parse(date))
    # train.sort_values(by = ['parsed_timestamp'])
    train["adj_value"] = train["value"].map(lambda value: float(elevation) + float(value))
    
    test = api_scraper.get_obs_for_link(link, start_date_test, end_date_test)
    test = pd.DataFrame(test, columns= ["value", "timestamp"])
    # test["parsed_timestamp"] = test["timestamp"].map(lambda date: date_parser.parse(date))
    # test.sort_values(by = ['parsed_timestamp'])
    test["adj_value"] = test["value"].map(lambda value: float(elevation) + float(value))
    
    return train, test 

In [9]:
dates = list(map(date_parser.parse, ["June 25 2019", "June 26 2019", "June 20 2019", "June 17 2019"]))
choices = ["Solomon Bridge Sea Level Sensor", "Hwy 21 at St Augustine Creek Level Sensor"]
train, test = get_train_test(dates[1], choices[1])

In [11]:
train

,value,timestamp,adj_value
0,-1.885,2019-06-18 00:02:20.541000+00:00,0.825
1,-1.865,2019-06-18 00:07:25.479000+00:00,0.845
2,-1.825,2019-06-18 00:17:34.826000+00:00,0.885
3,-1.807,2019-06-18 00:22:39.216000+00:00,0.903
4,-1.788,2019-06-18 00:27:43.410000+00:00,0.922
5,-1.771,2019-06-18 00:32:47.729000+00:00,0.939
6,-1.754,2019-06-18 00:37:51.849000+00:00,0.956
7,-1.740,2019-06-18 00:42:56.532000+00:00,0.970
8,-1.722,2019-06-18 00:48:00.810000+00:00,0.988
9,-1.707,2019-06-18 00:53:05.496000+00:00,1.003


In [ ]:
xdata_train = (train["timestamp"]
                .apply(lambda x: date_parser.parse(x))
                .apply(lambda x: mdates.date2num(x)))

#xdata_train = xdata_train.map(lambda x: (x - min(xdata_train))*100)
xdata_train = xdata_train.map(lambda x: (x)*100)
            
ydata_train = (train["adj_value"]
                .map(lambda y: (y)*100))

guess_amplitude = 80
guess_amplitude2 = 10
guess_periodicity = 0.122
guess_xshift = 7
guess_yshift = 10

guesses = [guess_amplitude, guess_periodicity, guess_xshift, guess_yshift]

guesses2 = [guess_amplitude, guess_amplitude2, guess_periodicity, guess_xshift, guess_yshift]

params, params_covariance = optimize.curve_fit(test_func2, xdata_train, ydata_train, method="lm",
                                               p0 = guesses2)

print(params)

plt.figure(figsize=(30, 20))
plt.scatter(xdata_train, ydata_train, color = "red", label='Data')
#plt.plot(xdata_train, test_func2(xdata_train, *guesses2),
 #        label='Guess')
plt.plot(xdata_train, test_func2(xdata_train, *params),
         label='Fitted function')
plt.legend(loc='best')

In [ ]:
xdata_test = test["timestamp"].apply(lambda x: date_parser.parse(x)).apply(lambda x: mdates.date2num(x))

ydata_test = test["adj_value"]
ydata_test = ydata_test.map(lambda y: (y)*100)
#xdata_test = xdata_test.map(lambda x: (x - min(xdata_test))*100)
xdata_test = xdata_test.map(lambda x: (x)*100)

plt.figure(figsize=(30, 20))
plt.scatter(xdata_test, ydata_test, color = "red", label='Data')
plt.plot(xdata_test, test_func2(xdata_test, *params),
         label='Fitted function')
plt.legend(loc='best')

In [ ]:
function_output = test_func2(xdata_test, *params)
residuals_squared = (function_output - ydata_test) ** 2
np.mean(residuals_squared)

In [ ]:
for index, elt in test.iterrows():
    print(elt)